# Logic of the Model for Energy Balance in Arc Discharge Plasma

Source : Plasma Engineering by Prof. keidar

##### Libraries

In [1]:
import numpy
import sympy
import math
import scipy
from scipy import constants

A convenient function of the scipy.constants library is that we can invoke constants in electromagnetic equations from the values built into the library. We can see what information is stored for each constant. For example:

In [2]:
scipy.constants.physical_constants["Stefan-Boltzmann constant"]

(5.670373e-08, 'W m^-2 K^-4', 2.1e-13)

## The Equations

First, the energy balance of the interelectrode plasma:

\begin{align}
I_{arc}U_{pl} + U_{c}I_{e}  = &    I_{arc} \big(2T_{e} +  U_{a} \big) + U_{iz}I_{ion} \\ & +  3 \frac{m_{e}}{m_{i}} \big( T_{e} - T_{a} \big) n_{e} \mu_{e} \pi R_{arc}^{2} L_{gap} \nonumber
\end{align}

This equation states that the Joule Heating  provided by the current should balance with the heat losses from the anode and ionization plus heat transfer between electrons to neutrals.

Next, a balance of energy at the cathode yields the following equation:

\begin{align}
I_{ion} \big( U_{c} + U_{iz} - \phi_{W} \big) = q_{conduction} + q_{convection}
\end{align}

Equation 2 above can be solved for cathode voltage as such:

\begin{align}
U_{c} = \frac{\pi R_{c}^{2} \bigg( \epsilon \sigma T_{c}^{4} + \frac{\big( T_{c} - T_{0} \big) \lambda }{\pi^{3/2} R_{c}} \bigg) }{I_{ion}} + \phi_{W} - U_{iz}
\end{align}

Additionally, anode potential drop can be given by:

\begin{align}
U_{a} =& - T_{e} ln \bigg( \frac{I_{th}}{I_{arc}} \bigg)
\end{align}

$I_{th}$ above represents the thermal current, and is define as:

\begin{align}
I_{th} = X
\end{align}

The equation for current continuity, which states that the total current is equal to the sum of the ion and electron currents:

\begin{align}
I_{arc} = I_{ion} + I_{e}
\end{align}

Ion and electron currents are also given explicitly:

\begin{align}
I_{e} =& \pi R_{c}^{2} \big( A T_{c}^{2} \big)  exp^{ \big( - \frac{e \phi_{W}}{k_{B} T_{c}} \big) } \\
I_{ion} =& \pi R_{c}^{2} \bigg( 0.6 e n_{e} \bigg[ \frac{T_{e}}{m_{i}} \bigg]^{0.5} \bigg)
\end{align}

The current continuity equation yields a non linear equation to solve for the cathode temperature:

\begin{align}
I_{arc} = \pi R_{c}^{2} \bigg( A T_{c}^{2}  exp^{ \big( - \frac{e \phi_{W}}{k_{B} T_{c}} \big)  } +  0.6 e n_{e} \bigg[ \frac{T_{e}}{m_{i}} \bigg]^{0.5} \bigg)
\end{align}


From the Saha equation, we can obtain number densities of ions, neutrals, and electrons of the individual species in the plasma. In this case, the elements are helium and carbon, so there will be two saha equations. These can be generalized considering l represents each individual species in the equation below:

\begin{align}
\frac{n_{i,l} n_{e}}{n_{o,l}} =& \bigg( \frac{2 \pi m_{e} k_{B} T_{e}}{h^{2}} \bigg)^{3/2} exp^{ \bigg( - \frac{E_{l}}{k_{b} T_{e}} \bigg) }
\end{align}


Finally, the anode temperature will depend on the electron temperature since heating of the anode by electrons creates a temperature rise in the anode, as we will later see.


At this point we can isolate variables to understand if the system of equations have closure. We can express the variable of interest in terms of what variables they depend on:

\begin{align}
I_{ion} = & f(T_{e}, n_{e}) \nonumber \\
I_{e} = & f(T_{c}) \nonumber \\
T_{c} = & f(T_{e}, n_{e}) \nonumber \\
T_{a} = & f(T_{e}) \nonumber \\
U_{c} = & f(T_{e}, n_{e}) \nonumber \\
n_{e} = & f(T_{e}) \nonumber
\end{align}

There is now a closure issue because there is no analytical equation that can be used to solve the electron temperature. Thus, we have to develop a solver that is based on initial guesses corresponding to the necessary inputs such as total pressure and electron temperature.

For now, we'll spend some time understanding how the equations work.

## Simulation for Given Arc Current and Experiment Pressure

Temperatures will vary based on on the input arc current and Pressure. For now, this notebook only considers arc current of 20 amps and Pressure of 300 Torr. 

The values for pressure and arc current are taken from results provided in Chapter 6.

##### Total Pressure - Experiment Input


In [3]:
Pt = 300 * 133.322 # torr to Pa
print('Assumed Pressure in Pascal: %.2f' % Pt)

Assumed Pressure in Pascal: 39996.60


##### Electron Temperature (Assumed)

In [4]:
Te_eV = 0.66 # given in eV
Te = Te_eV*(11604.52500617) # K
#Te = Te_eV*constants.e/constants.k # given in Kelvin

print('Assumed electron temperature in Kelvin: %.2f ' % Te)

Assumed electron temperature in Kelvin: 7658.99 


## Plasma Composition and the  Saha Equation

From the saha equations, we can compute the number density of ions and neutrals for each species, and also calculate the number density of electrons. However, the solution is not straightforward because we have five unknowns and 2 nonlinear equations.

From the equation of state, we can write an expression for the electron pressure:

\begin{equation}
P_{e} = n_{e}k_{B}T_{e} 
\end{equation}

Additionally, we can relate the total pressure to the electron pressure by the electron concentration:

\begin{equation}
C_{e} = \frac{P_{e}}{P_{T}}
\end{equation}

We can use several equations invoking ionization equilibrium as a form of chemical equilibrium. We can write concentrations for electrons and total number of neutrals and ions:

\begin{align}
C_{e} &= \frac{n_{e}}{n_{e} + n_{i,t} + n_{o,t} } \\
C_{o,t} &= \frac{n_{o,t}}{n_{e} + n_{i,t} + n_{o,t} } \\
C_{i,t} &= \frac{n_{i,t}}{n_{e} + n_{i,t} + n_{o,t} } 
\end{align}

Additionally, let's remember the quasi-neutral relationship:

\begin{equation}
n_{e} = n_{i,He} + n_{i,C}
\end{equation}



Process:

1. Give initial guess for number density of electrons. 
2. Find electron pressure
3. Find electron concentration
4. Find total number density of ions and neutrals
5. Find concentrations of ions and neutrals
6. Find individual number density of individual species

7. Check for convergence: How close do guessed values conserve atom conservation? How well do guessed values predict total Pressure?

### Electron number density  $ \Big( \frac{1}{m^3} \Big)$

In [5]:
ne_guess = 14e20

### Steps 2 through 5

In [6]:
Pe_guess = ne_guess*constants.k*Te
Ce_guess = Pe_guess/Pt

In [7]:
print('Concentration of electrons %.4f ' % Ce_guess)

Concentration of electrons 0.0037 


In [8]:
ni_t = ne_guess
no_t = (ne_guess - Ce_guess*(ne_guess+ni_t))/Ce_guess


In [9]:
print('number density of total ions %.4g ' % ni_t)

number density of total ions 1.4e+21 


In [10]:
print('number density of total neutrals %.4g ' % no_t)

number density of total neutrals 3.754e+23 


In [11]:
Ci = ni_t/(ne_guess+ni_t+no_t)
Co = no_t/(ne_guess+ni_t+no_t)

In [12]:
print('Concentration of Ions %.4f ' % Co)

Concentration of Ions 0.9926 


In [13]:
print('concentration of neutrals %.4g ' % Ci)

concentration of neutrals 0.003701 


In [14]:
Ct = Co + Ci + Ce_guess
print('Sum of concentrations should be close to 1 : %.6f ' % Ct)

Sum of concentrations should be close to 1 : 1.000000 


In [15]:
Nt = ni_t + no_t + ne_guess
print('Total number density : %.4g ' % Nt)

Total number density : 3.782e+23 


In [16]:
Pt_guess = Nt*constants.k*Te

In [17]:
Pt_guess
print('Total Pressure as a function of guessed values : %.4f ' % Pt_guess)

Total Pressure as a function of guessed values : 39996.6000 


### Ionization Energies of Helium and Carbon

In [18]:
#These are given in eV, https://dept.astro.lsa.umich.edu/~cowley/ionen.htm

#assume values for 1st spectra

E_He = 24.5874*constants.e  #ionization energy for helium in Joules
E_C = 11.2603*constants.e #ionization energy for carbon

# Values from http://periodictable.com/Elements/002/data.html

# I need to give ionization energies in Joules for the saha equation

# E_He =2372.3 #kJ/mol
# E_He2 = 5250.5 #kJ/mol

# E_C = 1086.5 #kJ/mol
# E_C2 = 2352.6 #kJ/mol
# E_C3 = 4620.5 #kJ/mol
# E_C4 = 6222.7 #kJ/mol
# E_C5 = 37831 #kJ/mol
# E_C6 = 47277 #kJ/mol

### Saha Equation - Right-Hand Side

In [19]:
A =  ( ( (2*numpy.pi*constants.m_e*constants.k*Te) / (constants.h**2) )**(3/2) ) *\
                 math.exp( (-E_He)/(constants.k*Te))
    
B =  ( ( (2*numpy.pi*constants.m_e*constants.k*Te) / (constants.h**2) )**(3/2) ) *\
                 math.exp( (-E_C) / (constants.k*Te))

In [20]:
A #this is the right-hand side for the helium eqn

107170757213.4718

In [21]:
B #this is the right-hand side for the carbon eqn

6.303709865846398e+19

### Find Number Densities of Individual Species

At this point we have four equations and four unknowns. We can do some math to solve for the unknowns. 

\begin{align}
\frac{n_{e}n_{i,He}}{n_{o,He}} &= A \\
\frac{n_{e}n_{i,C}}{n_{o,C}} &= B \\
n_{o,t} &= n_{o,He} + n_{o,C} \\
n_{i,t} &= n_{i,He} + n_{i,C} 
\end{align}

From equations 19 and 20 above, we can solve for number of ions in each species and use these expressions in equation 20 to solve for $n_{o,C} $ :

\begin{align}
n_{i,He} &= \frac{A n_{o,He}}{n_{e}} \nonumber \\
n_{i,C} &= \frac{B n_{o,C}}{n_{e}} \nonumber \\
n_{i,t} &=  \frac{A n_{o,He}}{n_{e}} + \frac{B n_{o,C}}{n_{e}} \nonumber \\
n_{o,C} &= \frac{n_{i,t} - n_{o,He}\frac{A}{n_{e}}}{1  - \frac{B}{n_{e}}}
\end{align}




Plugging equation 21 into equation 19, we can solve for one unknown:

\begin{equation}
n_{o,He} = \frac{n_{o,t} - n_{i,t}\frac{n_{e}}{B}}{1 - \frac{A}{B}}
\end{equation}


In [22]:
no_He = (no_t - (ne_guess*ni_t)/B)/(1 - A/B)

In [23]:
print('number density He neutrals %.4g ' % no_He)

number density He neutrals 3.443e+23 


Now, we can use quasi-neutrality from equation 16 to solve for the carbon ion number density:

In [24]:
no_C = no_t - no_He

In [25]:
print('number density C neutrals %.4g ' % no_C)

number density C neutrals 3.109e+22 


Similarly, we can use the saha equations to solve for the ions of each species:

In [26]:
ni_He = (no_He/ne_guess)*A

In [27]:
print('number density He ions %.4g ' % ni_He)

number density He ions 2.636e+13 


In [28]:
ni_C = (no_C/ne_guess)*B

In [29]:
print('number density C ions %.4g ' % ni_C)

number density C ions 1.4e+21 


In [30]:
Conservation = (ni_He + no_He)/(ni_C + no_C)

In [31]:
print(  'the ratio of helium to carbon partial pressures is %.2f' % Conservation )

the ratio of helium to carbon partial pressures is 10.60


In [32]:
Nt2 = (ni_He + ni_C) + (no_He + no_C) + ne_guess

n_He = ni_He + no_He
n_C = ni_C + no_C
C_He = n_He/Nt2
C_C = n_C/Nt2

In [33]:
Ct2 = Ce_guess + C_He + C_C
print('Sum of concentrations should be close to 1 : %.6f ' % Ct2)

Sum of concentrations should be close to 1 : 1.000000 


### Check Partial Pressures

In [34]:
P_He = C_He*Pt
P_C = C_C*Pt

In [35]:
P_He

36412.64964524645

In [36]:
P_C

3435.9091673886564

In [37]:
P_He/P_C

10.597675279326612

In [38]:
Pt - (P_He + P_C + Pe_guess )

0.0

Now that conservation of concentrations and pressure are satisfied we can move on to solve other properties

## Solve for Ion Current

### More Parameters

In [39]:
Iarc = 20

Rc = (12.5/2.) * 1./1000. # cathode radius [meters]
Ran = 6.35/2. * 1./1000.  #anode radius [meters]
Lgap = 3.5 * 1./1000. #interelectrode gap [meters]

Rarc = Ran*0.2*Iarc

Em_C = 0.70 #emissivity - graphite (0-3600 deg)

#Richardson Constant:
A = 1.2e6 #constant in electron current density eqn [A/(m^2 K^2)], pg.62

#Thermal Conductivity Carbon
lmbda_C = 1.7 #thermal conductivity carbon W m-1 K-1
#lmbda_C = 140 #Thermal Conductivity; 140 W/(m K), http://periodictable.com/Elements/006/data.html

n_e = ne_guess # number density of electrons

#Work Function, http://hyperphysics.phy-astr.gsu.edu/hbase/tables/photoelec.html
phi_C = 4.81*constants.e #work function of Carbon for photoelectric effect, eV to Joules
phi_C_eV = 4.81

#Ionization Potential of Carbon

#Initial Cathode and Anode Temperatures, assume room temp in Kelvin:
To = 298.15


### Mass of Helium and Carbon Ions [Kg]

In [40]:
UAMU = 1.660538921e-27 # Unified atomic mass unit in Kg

mi_He = UAMU*4.002602
mi_C = UAMU*12.0107

m_i_AMU = (mi_He + mi_C) # given in kilograms
#
print('mass of He and C Ions : %.4g ' % m_i_AMU)

mass of He and C Ions : 2.659e-26 


In [41]:
mi_C

1.9944234818454697e-26

In [42]:
scipy.constants.physical_constants["Stefan-Boltzmann constant"]

(5.670373e-08, 'W m^-2 K^-4', 2.1e-13)

### Ion Current - Check value!

Let's check the units for the ion current equation. We want the result given in amperes. In order for the units to give the expected result, we need to give the electron temperature in Joules.

In [43]:
I_ion = numpy.pi*(Rc**2)*\
0.6*(constants.e)*(n_e)*( ( (Te_eV*constants.e) / (m_i_AMU) ) **0.5)
print('Ion Current in Amperes : %.4g ' % I_ion)

Ion Current in Amperes : 32.94 


The resulting ion current does not correspond to the results provided on p. 317. The ion current should be about 80 percent of 20 amperes (which is the arc current). All of the other values used in this equation are in accordance with the values reported by the model results presented in chapter 6.

So, it appears that the mass of ions value given is incorrect. Using the reported values and solving for m_i, we should see what value it needs to be in order to satusfy the ion current reported in the book:

In [44]:
#This is the mass of ions i should get based on experimental results
m_i_sim = ((Te_eV*constants.e))/(((0.83*Iarc)/(numpy.pi*(Rc**2)*0.6*constants.e*n_e))**2)
m_i_sim

1.046736266307981e-25

In [45]:
#mass correction factor

mcf = m_i_sim/m_i_AMU
mcf

3.9364733701993044

In [46]:
#(m_i_sim/mi_C)*2

## Solve for Cathode Temperature

### Solve for Ion current with theoretical m_i

In [47]:
I_ion_sim = numpy.pi*(Rc**2)*0.6*(constants.e)*(n_e)*( ( (Te_eV*constants.e) / (m_i_sim) ) **0.5)
print('Ion Current with simulation m_i in Amperes : %.4g ' % I_ion_sim)

Ion Current with simulation m_i in Amperes : 16.6 


In [48]:
I_frac1 = I_ion_sim/Iarc
print('Ion Current/ Arc Current : %.4g ' % I_frac1)

Ion Current/ Arc Current : 0.83 


In [49]:
Tc = sympy.symbols('Tc')

From here, I'll apply the correction factor to the mass we found:

In [50]:
Tc_eqn = sympy.Eq( 20, numpy.pi*(Rc**2) *\
                  ( A*(Tc**2)*sympy.exp(-phi_C/(constants.k*Tc)) +\
                   0.6*(constants.e)*(n_e)*( ( (Te_eV*constants.e) / (m_i_AMU*mcf) ) **0.5) ) )

In [51]:
Tc_eqn

20 == 147.262155637022*Tc**2*exp(-55817.737846511/Tc) + 16.6

In [52]:
#Tcs = sympy.solve(Tc_eqn, Tc)
Tcs = sympy.nsolve(Tc_eqn, Tc, 2000)

In [53]:
Tc_sim = float(Tcs)

print('Cathode Temperature in Kelvin: %.4g ' % Tc_sim)

Cathode Temperature in Kelvin: 2838 


This cathode temperature does not match with the results presented in Figure 6.25 on page 320. The cathode temperature when Iarc = 20 is reported to be above 3200 Kelvin.

## Solve for Electron Current

In [54]:
I_e = numpy.pi*(Rc**2)*A*(Tc_sim**2)*numpy.exp(-phi_C/(constants.k*Tc_sim))
print('Electron Current in Amperes : %.4g ' % I_e)

Electron Current in Amperes : 3.4 


## Solve for Thermal Flux $ \frac{W}{m^2}$

In [55]:
q_con = ((Tc_sim-To)*lmbda_C)/(Rc*(numpy.pi**(3/2)) )
q_con

124051.97416184876

## Solve for Radiative Flux $ \frac{W}{m^2}$

In [56]:
q_rad = Em_C*constants.sigma*(Tc_sim**4)
q_rad

2573864.461249892

## Solve for Cathode Voltage

In [57]:
Uc_V = ((numpy.pi*(Rc**2))/(I_ion_sim))*( q_con + q_rad ) + (1/n_e)*( (phi_C_eV/constants.e) + (E_C/constants.e) )
Uc_eV = Uc_V * (constants.e*n_e)

In [58]:
print(' Cathode Voltage in Volts : %.4g ' % Uc_V)

 Cathode Voltage in Volts : 19.97 


In [59]:
print(' Cathode Voltage in Electron Volts : %.4g ' % Uc_eV)

 Cathode Voltage in Electron Volts : 4479 


## Solve for Anode Voltage

### Thermal Current

In [60]:
I_th = Iarc/(numpy.pi*(Rarc**2))*(numpy.pi*Ran**2)
I_th

1.2499999999999998

In [61]:
Ua_eV = - Te_eV * numpy.log(I_th/Iarc) #eV
Ua = Ua_eV*(n_e*constants.e)  #volts

In [62]:
print(' Anode Voltage Drop in Electron Volts : %.4g ' % Ua_eV)

 Anode Voltage Drop in Electron Volts : 1.83 


In [63]:
print(' Anode Voltage Drop in  Volts : %.4g ' % Ua)

 Anode Voltage Drop in  Volts : 410.5 


# Energy Balance Interelectrode Gap

In [64]:
# Voltage Drop in interelectrode gap:

#Upl = (Iarc*Lgap)/(constants.sigma*numpy.pi*(Rarc**2))
Upl_V = Iarc / ( constants.sigma*numpy.pi*(Rarc**2) * (Te)**4 )
Upl_V

0.00020229066221829552

In [65]:
J_Heating = Iarc*Upl_V + Uc_V*I_e
J_Heating

67.889373531946006

In [66]:
#Losses = 

## Working on loops:

In [67]:
from IPython.core.display import HTML
#css_file = '/Users/Luis/numerical-mooc/styles/numericalmoocstyle.css'
css_file = './numericalmoocstyle.css'
HTML(open(css_file, "r").read())